# The Performance Engineering Challenge: A Deep Dive

You have been given a simple kernel that runs in 147,734 cycles. The goal is to make it run in ~1,500 cycles.

That is a **100x speedup**.

How is that even possible? The answer lies in understanding how modern processors actually work, and then systematically exploiting every available resource. By the end of this notebook, you will understand not just *what* optimizations to apply, but *why* each one works and *how much* speedup it can deliver.

## What Does the Kernel Actually Do?

Before we can optimize anything, we need to deeply understand the problem. Let us trace through what the kernel computes.

In [ ]:
# Setup: Import the problem definitions
import sys
sys.path.insert(0, '..')

from problem import (
    Tree, Input, HASH_STAGES, myhash,
    reference_kernel, build_mem_image,
    SLOT_LIMITS, VLEN, SCRATCH_SIZE
)
import random

The kernel traverses a binary tree. But not just once - it does this for **256 independent traversals** running in parallel, each repeating for **16 rounds**.

Let us visualize the tree structure first.

In [ ]:
def visualize_binary_tree(height):
    """Show how a binary tree is stored as a flat array."""
    # Binary tree stored as array:
    # index 0 = root
    # left child of i = 2*i + 1
    # right child of i = 2*i + 2
    
    total_nodes = 2**(height+1) - 1
    print(f"Tree height: {height}")
    print(f"Total nodes: {total_nodes}")
    print()
    
    # Show tree structure for small height
    if height <= 3:
        for level in range(height + 1):
            start = 2**level - 1
            count = 2**level
            indices = list(range(start, start + count))
            spacing = " " * (2**(height - level + 1))
            nodes = spacing.join(f"[{i}]" for i in indices)
            print(f"Level {level}: {' ' * (2**(height - level))} {nodes}")
    else:
        print(f"(Tree too large to display, {height} levels)")
    
    print()
    print("Navigation:")
    print("  From node i:")
    print("    Left child  = 2*i + 1")
    print("    Right child = 2*i + 2")

visualize_binary_tree(3)

### The Traversal Algorithm

Each traversal works like this:

1. Start at some node with some value
2. XOR your value with the node's stored value
3. Hash the result (this is the expensive part)
4. If the hash is even, go left; if odd, go right
5. If you fall off the tree, wrap back to the root
6. Repeat

Let us trace through a single step:

In [ ]:
def trace_one_step(tree_values, current_idx, current_val):
    """Trace through one step of the traversal, showing all operations."""
    n_nodes = len(tree_values)
    
    print(f"Current position: node {current_idx}")
    print(f"Current value:    0x{current_val:08x}")
    print()
    
    # Step 1: Load node value
    node_val = tree_values[current_idx]
    print(f"1. Load node value: tree[{current_idx}] = 0x{node_val:08x}")
    
    # Step 2: XOR
    xored = current_val ^ node_val
    print(f"2. XOR with value:  0x{current_val:08x} ^ 0x{node_val:08x} = 0x{xored:08x}")
    
    # Step 3: Hash (expensive!)
    hashed = myhash(xored)
    print(f"3. Hash result:     myhash(0x{xored:08x}) = 0x{hashed:08x}")
    
    # Step 4: Compute next index
    is_even = (hashed % 2 == 0)
    direction = "LEFT" if is_even else "RIGHT"
    offset = 1 if is_even else 2
    next_idx = 2 * current_idx + offset
    print(f"4. Direction:       {'even' if is_even else 'odd'} -> go {direction}")
    print(f"   Next index:      2 * {current_idx} + {offset} = {next_idx}")
    
    # Step 5: Boundary check
    if next_idx >= n_nodes:
        print(f"5. Boundary check:  {next_idx} >= {n_nodes} -> WRAP to root")
        next_idx = 0
    else:
        print(f"5. Boundary check:  {next_idx} < {n_nodes} -> OK")
    
    print()
    print(f"Result: node {next_idx}, value 0x{hashed:08x}")
    return next_idx, hashed

# Create a small tree and trace one step
random.seed(42)
small_tree = Tree.generate(3)  # 15 nodes
trace_one_step(small_tree.values, 0, 0x12345678)

### The Hash Function: Where the Cycles Go

The hash function is not arbitrary - it has 6 stages, each with the same structure. Understanding this structure is crucial for optimization.

In [ ]:
def explain_hash_stages():
    """Show what each hash stage computes."""
    print("The hash function has 6 stages:")
    print()
    print("Each stage computes: a = (a op1 const1) op2 (a op3 shift_amount)")
    print()
    
    for i, (op1, val1, op2, op3, val3) in enumerate(HASH_STAGES):
        print(f"Stage {i}: a = (a {op1} 0x{val1:08x}) {op2} (a {op3} {val3})")
    
    print()
    print("Operations per stage:")
    print("  - 1 arithmetic/logic op (+ or ^) with constant")
    print("  - 1 shift operation")
    print("  - 1 combining operation (+ or ^)")
    print("  - 1 32-bit truncation (implicit)")
    print()
    print(f"Total: 6 stages x 3 operations = 18 ALU operations per hash")

explain_hash_stages()

### Counting Operations in the Inner Loop

Now let us count exactly how many operations happen in each iteration of the inner loop. This tells us the theoretical minimum work.

In [ ]:
def count_inner_loop_operations():
    """Count all operations in one inner loop iteration."""
    ops = {
        "load": 0,
        "store": 0,
        "alu": 0,
        "flow": 0
    }
    
    print("Operations per batch item per round:")
    print()
    
    # Load idx and val from input arrays
    print("Loading inputs:")
    print("  load idx from memory       (1 load)")
    print("  load val from memory       (1 load)")
    ops["load"] += 2
    
    # Load node value (dependent on idx)
    print("  compute idx address        (1 alu: +)")
    print("  load node_val from tree    (1 load)")
    ops["alu"] += 1
    ops["load"] += 1
    
    # XOR
    print("Computing hash input:")
    print("  val ^ node_val             (1 alu: ^)")
    ops["alu"] += 1
    
    # Hash - 6 stages, 3 ops each
    print("Hash function:")
    print("  6 stages x 3 ops           (18 alu)")
    ops["alu"] += 18
    
    # Compute next index
    print("Computing next index:")
    print("  val % 2                    (1 alu: %)")
    print("  val % 2 == 0               (1 alu: ==)")
    print("  select 1 or 2              (1 flow: select)")
    print("  2 * idx                    (1 alu: *)")
    print("  idx + offset               (1 alu: +)")
    ops["alu"] += 4
    ops["flow"] += 1
    
    # Boundary check
    print("Boundary check:")
    print("  idx < n_nodes              (1 alu: <)")
    print("  select idx or 0            (1 flow: select)")
    ops["alu"] += 1
    ops["flow"] += 1
    
    # Store results
    print("Storing outputs:")
    print("  compute store addresses    (2 alu: +)")
    print("  store idx                  (1 store)")
    print("  store val                  (1 store)")
    ops["alu"] += 2
    ops["store"] += 2
    
    print()
    print("Summary:")
    for op, count in ops.items():
        print(f"  {op}: {count}")
    print(f"  TOTAL: {sum(ops.values())} operations")
    
    return ops

inner_ops = count_inner_loop_operations()

## The Full Scale: Why 147,734 Cycles?

Now we can understand why the baseline is so slow. The test case processes:
- 256 batch items
- 16 rounds each
- ~30 operations per (item, round) pair

Let us calculate:

In [ ]:
def analyze_baseline_performance():
    """Understand why the baseline takes 147,734 cycles."""
    
    BATCH_SIZE = 256
    ROUNDS = 16
    BASELINE_CYCLES = 147734
    
    # Operations per inner loop iteration
    ops_per_iteration = 30  # approximately
    
    total_iterations = BATCH_SIZE * ROUNDS
    total_operations = total_iterations * ops_per_iteration
    
    print("Test case parameters:")
    print(f"  Batch size:  {BATCH_SIZE}")
    print(f"  Rounds:      {ROUNDS}")
    print(f"  Tree height: 10 (2047 nodes)")
    print()
    
    print(f"Total inner loop iterations: {BATCH_SIZE} x {ROUNDS} = {total_iterations}")
    print(f"Operations per iteration: ~{ops_per_iteration}")
    print(f"Total operations: ~{total_operations:,}")
    print()
    
    print(f"Baseline cycles: {BASELINE_CYCLES:,}")
    cycles_per_iteration = BASELINE_CYCLES / total_iterations
    print(f"Cycles per iteration: {cycles_per_iteration:.1f}")
    print()
    
    print("The baseline uses one instruction per cycle.")
    print(f"That is about {cycles_per_iteration:.0f} cycles for {ops_per_iteration} operations.")
    print("Extra cycles come from address calculations and setup.")

analyze_baseline_performance()

## The Machine: What Resources Do We Have?

Now comes the key insight. The baseline is slow because it uses only a tiny fraction of available resources. Let us see what the machine offers.

In [ ]:
def show_machine_resources():
    """Display all available engine slots."""
    
    print("Available resources per cycle:")
    print("=" * 50)
    print()
    
    for engine, slots in SLOT_LIMITS.items():
        if engine == "debug":
            continue  # Not relevant for performance
        bar = "#" * slots
        print(f"{engine:8s}: [{bar:<12}] {slots} slots")
    
    print()
    print("Key facts:")
    print(f"  - VLEN = {VLEN} (vector operations process {VLEN} elements)")
    print(f"  - Scratch space: {SCRATCH_SIZE} words of fast memory")
    print()
    
    # Calculate theoretical throughput
    scalar_alu_per_cycle = SLOT_LIMITS["alu"]
    vector_alu_per_cycle = SLOT_LIMITS["valu"] * VLEN
    loads_per_cycle = SLOT_LIMITS["load"]
    stores_per_cycle = SLOT_LIMITS["store"]
    
    print("Theoretical throughput per cycle:")
    print(f"  - Scalar ALU operations: {scalar_alu_per_cycle}")
    print(f"  - Vector ALU elements:   {vector_alu_per_cycle} ({SLOT_LIMITS['valu']} ops x {VLEN} elements)")
    print(f"  - Memory loads:          {loads_per_cycle}")
    print(f"  - Memory stores:         {stores_per_cycle}")

show_machine_resources()

### The Baseline Uses Almost Nothing

The baseline implementation (shown below) uses **one slot per cycle**. That means it wastes:
- 11 out of 12 scalar ALU slots
- All 6 vector ALU slots
- 1 out of 2 load slots
- 1 out of 2 store slots

In [ ]:
def show_baseline_utilization():
    """Visualize how poorly the baseline uses resources."""
    
    print("Baseline resource utilization per cycle:")
    print("=" * 50)
    print()
    
    # In the baseline, each instruction uses exactly 1 slot
    utilization = {
        "alu": 1,   # Uses 1 of 12
        "valu": 0,  # Uses 0 of 6
        "load": 1,  # Uses 1 of 2
        "store": 1, # Uses 1 of 2
        "flow": 1   # Uses 1 of 1
    }
    
    for engine, slots in SLOT_LIMITS.items():
        if engine == "debug":
            continue
        used = utilization.get(engine, 0)
        pct = (used / slots) * 100
        used_bar = "#" * used
        unused_bar = "." * (slots - used)
        print(f"{engine:8s}: [{used_bar}{unused_bar:<12}] {used}/{slots} = {pct:5.1f}%")
    
    print()
    print("The baseline wastes most of the machine's capacity!")
    print("This is why 100x speedup is possible.")

show_baseline_utilization()

## The Critical Bottleneck: Memory, Not Compute

Before we dive into optimizations, we need to identify what will ultimately limit our performance. This is crucial because it tells us where to focus our efforts.

Look at the slot counts again:
- **12 ALU slots** + **6 vALU slots** (each processing 8 elements) = massive compute capacity
- **2 load slots** + **2 store slots** = very limited memory bandwidth

The kernel does 8 scattered tree loads per vector group. With only 2 load slots, that is **4 cycles minimum** just for tree loads, no matter how fast our compute is.

This has a profound implication: **memory is the bottleneck, not compute**. The hash function takes 12 cycles due to dependencies, but we only need 4 cycles of loads. If we can overlap the loads with the hash computation, memory becomes the limiting factor.

Target milestone breakdown (keep these in mind):
- **Scalar baseline**: 147,734 cycles (1 op per cycle)
- **Vectorized (8x SIMD)**: ~7,700 cycles (8 items at once)
- **+ VLIW packing**: ~3,500 cycles (multiple ops per cycle)
- **+ Software pipelining**: ~1,500 cycles (overlap memory and compute)

Each optimization builds on the previous. Let us see how.

## Optimization Strategy 1: Vectorization (SIMD)

The most important observation: **the 256 batch items are completely independent**. We can process 8 of them simultaneously using vector operations.

This is not a new idea - you learned about SIMD in the parallelism notebook. The key insight here is that SIMD works perfectly for this problem because:

1. Each batch item follows the same operations
2. There are no dependencies between batch items
3. The batch size (256) is a multiple of VLEN (8)

In [ ]:
def analyze_vectorization_opportunity():
    """Show how vectorization transforms the computation."""
    
    BATCH_SIZE = 256
    
    print("Scalar approach (baseline):")
    print(f"  Process 1 batch item at a time")
    print(f"  {BATCH_SIZE} iterations per round")
    print()
    
    print(f"Vector approach (VLEN = {VLEN}):")
    print(f"  Process {VLEN} batch items at a time")
    print(f"  {BATCH_SIZE // VLEN} iterations per round")
    print()
    
    speedup = BATCH_SIZE / (BATCH_SIZE // VLEN)
    print(f"Potential speedup from vectorization: {speedup}x")
    print()
    
    print("What changes:")
    print("  Scalar:  val = myhash(val ^ node_val)   // 1 value")
    print("  Vector:  vals[0:8] = myhash(vals[0:8] ^ node_vals[0:8])  // 8 values")

analyze_vectorization_opportunity()

### The Catch: Scattered Memory Access

There is one complication. While the batch `values` array is contiguous (we can use `vload`/`vstore`), the tree nodes we need to load are **scattered** - each batch item might be at a different tree position.

This means we cannot use `vload` for tree values. We need 8 separate scalar loads.

In [ ]:
def explain_scattered_access():
    """Show why tree access is scattered."""
    
    print("Memory access patterns:")
    print()
    print("Batch indices (contiguous - can use vload):")
    print("  inp_indices[0], inp_indices[1], ..., inp_indices[7]")
    print("  These are consecutive memory locations")
    print()
    print("Batch values (contiguous - can use vload/vstore):")
    print("  inp_values[0], inp_values[1], ..., inp_values[7]")
    print("  These are consecutive memory locations")
    print()
    print("Tree values (SCATTERED - must use scalar loads):")
    print("  tree[indices[0]], tree[indices[1]], ..., tree[indices[7]]")
    print("  Example: tree[0], tree[15], tree[7], tree[3], tree[1023], ...")
    print("  These are NON-consecutive memory locations")
    print()
    print("Impact on slots:")
    print(f"  We have only 2 load slots per cycle")
    print(f"  Loading {VLEN} scattered tree values needs {VLEN} loads")
    print(f"  That takes {VLEN // 2} cycles just for tree loads")

explain_scattered_access()

### Vectorization Cycle Count Estimate

Let us estimate how many cycles a vectorized implementation would take.

In [ ]:
def estimate_vectorized_cycles():
    """Estimate cycles after vectorization."""
    
    BATCH_SIZE = 256
    ROUNDS = 16
    BASELINE = 147734
    
    vector_groups = BATCH_SIZE // VLEN  # 32 groups of 8
    
    print("Per vector group (8 batch items):")
    print()
    
    # Count cycles for each operation
    cycles = {}
    
    # Load batch indices - vload takes 1 slot, 1 cycle
    cycles["load batch indices"] = 1
    print(f"  Load 8 indices (vload):        1 cycle")
    
    # Load batch values - vload takes 1 slot, 1 cycle  
    cycles["load batch values"] = 1
    print(f"  Load 8 values (vload):         1 cycle")
    
    # Load tree values - 8 scattered loads, 2 per cycle
    tree_load_cycles = (VLEN + 1) // 2  # ceiling division
    cycles["load tree values"] = tree_load_cycles
    print(f"  Load 8 tree values (scattered): {tree_load_cycles} cycles")
    
    # XOR - 1 valu op
    cycles["xor"] = 1
    print(f"  XOR values with tree values:   1 cycle")
    
    # Hash - 6 stages, each stage has a dependency chain
    # Each stage: compute tmp1 and tmp2 in parallel (1 cycle), then combine (1 cycle)
    # So each stage takes 2 cycles minimum
    hash_cycles = 12  # 6 stages x 2 cycles per stage (dependency limited)
    cycles["hash"] = hash_cycles
    print(f"  Hash 6 stages (2 cycles each): {hash_cycles} cycles")
    
    # Compute next index - several valu ops
    cycles["next index"] = 2
    print(f"  Compute next index:            ~2 cycles")
    
    # Store - vstore for values, scattered for indices
    cycles["store"] = tree_load_cycles + 1
    print(f"  Store results:                 ~{cycles['store']} cycles")
    
    total_per_group = sum(cycles.values())
    print()
    print(f"  Total per vector group: ~{total_per_group} cycles")
    print()
    
    total = vector_groups * ROUNDS * total_per_group
    print(f"Total estimated cycles:")
    print(f"  {vector_groups} groups x {ROUNDS} rounds x {total_per_group} cycles = {total:,}")
    print()
    print(f"Speedup over baseline: {BASELINE / total:.1f}x")
    print()
    print("This is still far from the 1,500 cycle target!")
    print("But wait - we identified the bottleneck earlier: MEMORY.")
    print("The hash takes 12 cycles, but we only have 4 cycles of loads.")
    print("Can we overlap them?")
    
    return total

vectorized_estimate = estimate_vectorized_cycles()

## Optimization Strategy 2: Instruction Packing (VLIW)

Recall from the parallelism notebook: VLIW lets us issue multiple independent operations in the same cycle. The baseline issues ONE operation per cycle. We can do much better.

The key is to find operations that:
1. Do not depend on each other
2. Use different engine slots (or fit within slot limits)
3. Are needed at approximately the same time

In [ ]:
def show_vliw_packing_example():
    """Demonstrate how to pack multiple operations per cycle."""
    
    print("Baseline (1 op per cycle):")
    print("  Cycle 1: load indices       // load engine")
    print("  Cycle 2: load values        // load engine")
    print("  Cycle 3: compute addr       // alu engine")
    print("  Cycle 4: load tree[0]       // load engine")
    print("  Cycle 5: compute addr       // alu engine")
    print("  Cycle 6: load tree[1]       // load engine")
    print("  ... and so on")
    print()
    
    print("Packed (multiple ops per cycle):")
    print("  Cycle 1: { load: [vload indices, vload values] }")
    print("  Cycle 2: { load: [load tree[0], load tree[1]], alu: [compute addrs...] }")
    print("  Cycle 3: { load: [load tree[2], load tree[3]], alu: [xor vals...] }")
    print("  ... operations overlap")
    print()
    
    print("VLIW instruction format example:")
    example_instr = {
        "valu": [("^", 40, 24, 32), ("+", 48, 40, 56)],  # 2 vector ops
        "load": [("load", 100, 101), ("load", 102, 103)], # 2 loads
        "alu": [("+", 10, 11, 12)]  # 1 scalar op
    }
    print(f"  {example_instr}")
    print()
    print("This single instruction does:")
    print("  - 2 vector XOR/ADD operations (16 elements total)")
    print("  - 2 memory loads")
    print("  - 1 scalar addition")
    print("  All in ONE cycle!")

show_vliw_packing_example()

### The Hash Pipeline Opportunity

The hash function has 6 stages that must be done sequentially (each stage depends on the previous). But we can still pack operations *within* each stage, and we can overlap hash computation with memory operations.

Even better: we can be processing round N+1's loads while computing round N's hash!

## Optimization Strategy 3: Software Pipelining

**Why do we need this?** VLIW packing got us to ~3,500 cycles. But there is still a fundamental problem: we process each vector group completely before starting the next one. The hash takes 12 cycles, but we only need 4 cycles of loads. That means the load slots sit idle for 8 cycles!

**The key insight:** Memory is the bottleneck (we established this at the start). The hash computation is slow due to dependencies, but it uses vALU slots. The loads use load slots. These are DIFFERENT resources. We can use them simultaneously!

**Target:** Get from ~3,500 cycles to ~1,500 cycles (about 2.3x improvement).

This is called **software pipelining** - manually scheduling operations so that while one vector group is computing its hash, the next group is loading its data.

In [ ]:
def illustrate_software_pipeline():
    """Show how pipelining overlaps different stages with concrete cycles."""
    
    print("SOFTWARE PIPELINING: The Key to 100x Speedup")
    print("=" * 60)
    print()
    print("The idea: While computing hash for group N, load data for group N+1.")
    print("This hides memory latency behind computation.")
    print()
    
    print("CONCRETE CYCLE-BY-CYCLE SCHEDULE (simplified):")
    print("-" * 60)
    print()
    print("Legend: G0 = Group 0, G1 = Group 1, etc.")
    print("        L = Load phase, H = Hash phase, S = Store phase")
    print()
    print("Cycle  | Load Slots (2)        | vALU Slots (6)          | Notes")
    print("-------|----------------------|-------------------------|------------------")
    print("  1    | vload G0 idx,val     | --                      | Start G0")
    print("  2    | load G0 tree[0,1]    | --                      | Scattered loads")
    print("  3    | load G0 tree[2,3]    | --                      |")
    print("  4    | load G0 tree[4,5]    | XOR G0                  | Compute can start")
    print("  5    | load G0 tree[6,7]    | Hash G0 stg0 (ops 1,2)  |")
    print("  6    | vload G1 idx,val     | Hash G0 stg0 (op 3)     | G1 loads begin!")
    print("  7    | load G1 tree[0,1]    | Hash G0 stg1            |")
    print("  8    | load G1 tree[2,3]    | Hash G0 stg1            |")
    print("  9    | load G1 tree[4,5]    | Hash G0 stg2, XOR G1    | G1 compute starts")
    print(" 10    | load G1 tree[6,7]    | Hash G0 stg2, Hash G1   |")
    print(" 11    | vload G2 idx,val     | Hash G0 stg3, Hash G1   | G2 loads begin!")
    print(" 12    | load G2 tree[0,1]    | Hash G0 stg3, Hash G1   |")
    print(" ...   | (continues)          | (multiple in flight)    |")
    print()
    
    print("KEY INSIGHT: After the pipeline fills (~5 cycles), we process")
    print("one vector group every ~3 cycles in steady state!")
    print()
    
    # Calculate the math
    BATCH_SIZE = 256
    ROUNDS = 16
    vector_groups = BATCH_SIZE // VLEN
    total_iterations = vector_groups * ROUNDS
    
    # Without pipelining
    cycles_per_group_unpipelined = 4 + 12 + 3  # load + hash + store/index
    total_unpipelined = total_iterations * cycles_per_group_unpipelined
    
    # With pipelining - memory bound at ~3 cycles per group in steady state
    # (4 scattered loads need 4 cycles, but we have 2 load slots, so need 4/2=2 cycles
    # for tree loads, plus 1 for vloads = 3 cycles per group when pipelined)
    cycles_per_group_pipelined = 3
    pipeline_fill = 5
    pipeline_drain = 5
    total_pipelined = total_iterations * cycles_per_group_pipelined + pipeline_fill + pipeline_drain
    
    print(f"Without pipelining: {total_iterations} groups x {cycles_per_group_unpipelined} = {total_unpipelined:,} cycles")
    print(f"With pipelining:    {total_iterations} groups x {cycles_per_group_pipelined} + overhead = {total_pipelined:,} cycles")
    print()
    print(f"Pipelining speedup: {total_unpipelined / total_pipelined:.1f}x")
    print()
    print("This gets us into the ~1,500 cycle range!")

illustrate_software_pipeline()

## Understanding the Instruction Format

Before you start implementing, you need to understand how instructions are encoded. Let us examine the actual format used by the `KernelBuilder` class.

In [ ]:
def explain_instruction_format():
    """Explain how to write instructions for the machine."""
    
    print("Instruction Bundle Format:")
    print("=" * 50)
    print()
    print("Each cycle executes ONE instruction bundle.")
    print("A bundle is a dict mapping engine names to lists of operations.")
    print()
    print("Example bundle:")
    example = {
        "valu": [("^", 40, 24, 32), ("+", 48, 40, 56)],
        "load": [("const", 100, 0x7ED55D16), ("load", 102, 103)],
        "alu": [("+", 10, 11, 12)]
    }
    for engine, ops in example.items():
        print(f"  \"{engine}\": {ops}")
    print()
    
    print("Operation tuple format:")
    print("  (opcode, dest, src1, src2)   for ALU/vALU ops")
    print("  (\"load\", dest, addr)          for load")
    print("  (\"vload\", dest, addr)         for vector load (addr is scalar)")
    print("  (\"store\", addr, src)          for store")
    print("  (\"vstore\", addr, src)         for vector store")
    print("  (\"const\", dest, value)        for loading constants")
    print("  (\"select\", dest, cond, t, f)  for conditional select")
    print("  (\"vselect\", dest, cond, t, f) for vector conditional select")
    print()
    
    print("All numbers are scratch space addresses (0 to 1535).")
    print("For vector operations, address N refers to N, N+1, ..., N+7.")
    print()
    
    print("Available ALU operations:")
    ops = ["+", "-", "*", "//", "^", "&", "|", "<<", ">>", "%", "<", "=="]
    print(f"  {ops}")

explain_instruction_format()

### Scratch Space Management

The scratch space is your "register file" - all operations read from and write to scratch addresses. You need to manage this space carefully.

## Minimal Working Example: From Scalar to Vectorized

This is the most important section for understanding the transformation. We show exactly how baseline scalar code becomes optimized vectorized VLIW code.

In [ ]:
def show_code_transformation():
    """Show the actual instruction transformation from scalar to vectorized."""
    
    print("TRANSFORMATION EXAMPLE: Loading and XORing batch data")
    print("=" * 70)
    print()
    
    print("BASELINE (scalar, 1 op per instruction):")
    print("-" * 70)
    baseline_instrs = [
        '{"load": [("const", tmp1, 0)]}           # tmp1 = 0 (batch index)',
        '{"alu":  [("+", addr, inp_indices_p, tmp1)]}  # addr = inp_indices_p + 0',
        '{"load": [("load", idx, addr)]}              # idx = mem[addr]',
        '{"alu":  [("+", addr, inp_values_p, tmp1)]}  # addr = inp_values_p + 0',
        '{"load": [("load", val, addr)]}              # val = mem[addr]',
        '{"alu":  [("+", tree_addr, forest_p, idx)]}  # tree_addr = forest_p + idx',
        '{"load": [("load", node_val, tree_addr)]}    # node_val = mem[tree_addr]',
        '{"alu":  [("^", val, val, node_val)]}        # val = val ^ node_val',
        '# ... repeat for batch items 1, 2, 3, 4, 5, 6, 7 ...',
    ]
    for i, instr in enumerate(baseline_instrs, 1):
        print(f"  Cycle {i}: {instr}")
    print()
    print(f"  Total for 8 items: 8 x 8 = 64+ cycles (plus hash overhead)")
    print()
    
    print("OPTIMIZED (vectorized VLIW, multiple ops per instruction):")
    print("-" * 70)
    optimized_instrs = [
        ('{"alu": [("+", idx_base, inp_indices_p, batch_offset),\n'
         '         ("+", val_base, inp_values_p, batch_offset)]}'),
        '{"load": [("vload", v_idx, idx_base), ("vload", v_val, val_base)]}',
        '{"valu": [("+", v_tree_addr, v_forest_p, v_idx)]}',
        '{"alu": [("+", addr0, v_tree_addr+0, zero), (+, addr1, v_tree_addr+1, zero), ...]}',
        '{"load": [("load", v_node+0, addr0), ("load", v_node+1, addr1)]}',
        '{"load": [("load", v_node+2, addr2), ("load", v_node+3, addr3)]}',
        '{"load": [("load", v_node+4, addr4), ("load", v_node+5, addr5)]}',
        '{"load": [("load", v_node+6, addr6), ("load", v_node+7, addr7)]}',
        '{"valu": [("^", v_val, v_val, v_node)]}',
    ]
    for i, instr in enumerate(optimized_instrs, 1):
        print(f"  Cycle {i}: {instr}")
    print()
    print(f"  Total for 8 items: 9 cycles (not 64!)")
    print()
    
    print("KEY TRANSFORMATIONS APPLIED:")
    print("  1. vload/vstore: Load 8 contiguous values in 1 cycle (vs 8 cycles)")
    print("  2. valu: XOR 8 values in 1 cycle (vs 8 cycles)")
    print("  3. VLIW packing: Compute both addresses in cycle 1 (vs 2 cycles)")
    print("  4. Parallel loads: 2 scattered loads per cycle (vs 1)")
    print()
    
    print("WHAT THIS LOOKS LIKE IN THE KERNEL BUILDER:")
    print("-" * 70)
    print('''
# Vector load batch data (2 vloads in 1 instruction)
instrs.append({
    "load": [
        ("vload", v_idx, idx_base),    # Load 8 indices
        ("vload", v_val, val_base)     # Load 8 values
    ]
})

# Compute tree addresses (vector add)
instrs.append({
    "valu": [("+", v_tree_addr, v_forest_p, v_idx)]
})

# Scattered tree loads (2 per cycle, takes 4 cycles for 8 loads)
for i in range(0, VLEN, 2):
    instrs.append({
        "load": [
            ("load", v_node + i, addr_scalars[i]),
            ("load", v_node + i + 1, addr_scalars[i + 1])
        ]
    })

# Vector XOR (all 8 in 1 cycle)
instrs.append({
    "valu": [("^", v_val, v_val, v_node)]
})
''')

show_code_transformation()

## The Critical Bottleneck (Recap)

We front-loaded this insight at the beginning, but now that we understand all the optimizations, let us verify our bottleneck analysis with concrete numbers.

In [ ]:
def identify_bottleneck():
    """Reinforce the memory bottleneck with concrete numbers."""
    
    BATCH_SIZE = 256
    ROUNDS = 16
    vector_groups = BATCH_SIZE // VLEN
    total_iterations = vector_groups * ROUNDS
    
    print("MEMORY BOTTLENECK ANALYSIS (RECAP)")
    print("=" * 50)
    print()
    print("We established early that memory is the bottleneck.")
    print("Now let us quantify exactly how tight this constraint is.")
    print()
    
    # Irreducible memory operations
    tree_loads = VLEN  # Must load 8 tree values
    batch_loads = 2    # 1 vload for indices, 1 for values
    batch_stores = 2   # 1 vstore each
    
    print("Per vector group (irreducible work):")
    print(f"  Scattered tree loads:          {tree_loads}")
    print(f"  Contiguous batch loads:        {batch_loads} (vload)")
    print(f"  Contiguous batch stores:       {batch_stores} (vstore)")
    print()
    
    # Memory time calculation
    load_slots = 2
    store_slots = 2
    
    # Tree loads are scattered (cannot use vload), need individual loads
    tree_load_cycles = (tree_loads + load_slots - 1) // load_slots  # = 4 cycles
    batch_load_cycles = (batch_loads + load_slots - 1) // load_slots  # = 1 cycle
    total_load_cycles = tree_load_cycles + batch_load_cycles
    
    print(f"Load cycles needed per group:")
    print(f"  Tree loads (scattered):   {tree_loads} / {load_slots} slots = {tree_load_cycles} cycles")
    print(f"  Batch loads (contiguous): {batch_loads} / {load_slots} slots = {batch_load_cycles} cycle")
    print(f"  Total load cycles:        {total_load_cycles} cycles")
    print()
    
    # Theoretical minimum
    theoretical_min = total_iterations * tree_load_cycles
    print(f"THEORETICAL MINIMUM (memory-bound):")
    print(f"  {total_iterations} groups x {tree_load_cycles} cycles = {theoretical_min:,} cycles")
    print()
    
    # With stores overlapped
    print("But stores can overlap with next iteration's loads!")
    print("With perfect pipelining, we approach the load-limited minimum.")
    print()
    
    # Hash comparison
    print("COMPUTE vs MEMORY comparison:")
    print(f"  Hash computation: 12 cycles (dependency limited)")
    print(f"  Tree loads:       {tree_load_cycles} cycles (bandwidth limited)")
    print(f"  Ratio: 12 / {tree_load_cycles} = {12/tree_load_cycles:.1f}x")
    print()
    print("We have 3x more compute time than load time!")
    print("This is WHY pipelining works - we hide loads behind compute.")
    print()
    print("TARGET: ~1,500 cycles (includes pipeline fill/drain overhead)")

identify_bottleneck()

In [ ]:
def estimate_optimized_cycles():
    """Estimate achievable cycles with full optimization."""
    
    BATCH_SIZE = 256
    ROUNDS = 16
    BASELINE = 147734
    TARGET = 1500
    
    vector_groups = BATCH_SIZE // VLEN  # 32
    total_iterations = vector_groups * ROUNDS  # 512
    
    print("Key bottleneck analysis:")
    print("=" * 50)
    print()
    
    # Memory bottleneck
    print("Memory operations per iteration:")
    loads_per_iter = 8 + 2  # 8 scattered tree loads, 2 vloads for batch data
    stores_per_iter = 8 + 2  # 8 scattered stores (if needed), 2 vstores
    print(f"  Loads: ~{loads_per_iter} (8 tree + 2 batch)")
    print(f"  Stores: ~{stores_per_iter} (could optimize)")
    print(f"  With 2 load slots: {loads_per_iter // 2} cycles for loads")
    print()
    
    # Compute bottleneck
    print("Compute operations per iteration:")
    print(f"  Hash: 6 stages x 2 cycles = 12 cycles (dependency limited)")
    print(f"  Other: ~4 cycles (XOR, index calc, etc.)")
    print()
    
    # With pipelining
    print("With software pipelining:")
    print("  Can overlap loads with computation")
    print("  Effective cycles per iteration: ~3-5 (in steady state)")
    print()
    
    # Rough estimates
    cycles_per_iter_basic = 16  # Conservative
    cycles_per_iter_pipelined = 4  # Aggressive
    
    basic_total = total_iterations * cycles_per_iter_basic
    pipelined_total = total_iterations * cycles_per_iter_pipelined + 50  # Overhead
    
    print("Estimated total cycles:")
    print(f"  Conservative (no deep pipelining): ~{basic_total:,}")
    print(f"  Aggressive (full pipelining):      ~{pipelined_total:,}")
    print()
    print(f"  Target: {TARGET:,}")
    print(f"  Baseline: {BASELINE:,}")
    print()
    
    print("To reach ~1500 cycles:")
    cycles_per_iter_needed = (TARGET - 50) / total_iterations
    print(f"  Need ~{cycles_per_iter_needed:.1f} cycles per iteration")
    print("  This requires aggressive pipelining and optimal scheduling.")

estimate_optimized_cycles()

## Running and Testing Your Implementation

Here is how to test your optimizations.

In [ ]:
def show_testing_workflow():
    """Explain how to test implementations."""
    
    print("Testing workflow:")
    print("=" * 50)
    print()
    print("1. Quick cycle count check:")
    print("   python perf_takehome.py Tests.test_kernel_cycles")
    print()
    print("2. Full validation (required for submission):")
    print("   python tests/submission_tests.py")
    print()
    print("3. Visual debugging with Perfetto trace:")
    print("   python perf_takehome.py Tests.test_kernel_trace")
    print("   python watch_trace.py  # Opens browser")
    print()
    print("4. Verify tests folder unchanged:")
    print("   git diff origin/main tests/")
    print()
    print("Performance thresholds (from submission_tests.py):")
    print("  - Pass all tests for correctness")
    print("  - Target: < 1500 cycles for impressive result")
    print("  - Best human: 1790 cycles (in 2 hours)")
    print("  - Best AI: 1363 cycles (Claude Opus 4.5)")

show_testing_workflow()

## Practice: Understanding the Baseline Code

Let us look at how the baseline implementation works. Understanding this is essential before you can improve it.

In [ ]:
def test_understand_baseline_structure():
    """Verify understanding of baseline code structure."""
    
    # The baseline is in perf_takehome.py, in KernelBuilder.build_kernel()
    # It generates instructions for a scalar implementation
    
    # Question: How many instruction bundles does the baseline generate
    # for the full test case (256 batch, 16 rounds)?
    
    # Let's count:
    # - Setup instructions: ~20 (loading constants, addresses)
    # - Per (batch_item, round) pair:
    #   - Load idx, load val: ~4 instructions
    #   - Load node_val: ~2 instructions  
    #   - XOR: 1 instruction
    #   - Hash (6 stages x 3 ops): ~18 instructions
    #   - Index calculation: ~5 instructions
    #   - Boundary check: ~2 instructions
    #   - Stores: ~4 instructions
    #   Total: ~36 instructions per inner iteration
    
    BATCH_SIZE = 256
    ROUNDS = 16
    
    setup_instrs = 20
    per_iteration = 36
    total_iterations = BATCH_SIZE * ROUNDS
    
    estimated_total = setup_instrs + per_iteration * total_iterations
    
    print("Baseline instruction count estimate:")
    print(f"  Setup: ~{setup_instrs}")
    print(f"  Per iteration: ~{per_iteration}")
    print(f"  Iterations: {total_iterations}")
    print(f"  Total: ~{estimated_total:,}")
    print()
    print(f"  Actual baseline cycles: 147,734")
    print(f"  Our estimate: {estimated_total:,}")
    print(f"  (Close enough - differences from address calculations and debug ops)")
    
    # Assert our understanding is in the right ballpark
    assert 100000 < estimated_total < 200000, "Estimate way off - review the code"
    print()
    print("TEST PASSED: Understanding of baseline structure is correct.")

test_understand_baseline_structure()

## Practice: Identifying Parallelism

Can you identify which operations in the inner loop can be parallelized?

In [ ]:
def test_identify_parallelism():
    """Test understanding of parallelism opportunities."""
    
    # Consider processing batch items 0-7 simultaneously
    # Which of these operations can be done in parallel across all 8?
    
    operations = {
        "load_indices": True,      # Yes - independent memory locations
        "load_values": True,       # Yes - independent memory locations
        "load_tree_values": True,  # Yes - though scattered, still independent
        "xor_with_node": True,     # Yes - pure SIMD
        "hash_computation": True,  # Yes - same ops on independent data
        "index_calculation": True, # Yes - same ops on independent data
        "store_results": True,     # Yes - independent memory locations
    }
    
    print("Parallelism analysis (batch items 0-7):")
    print()
    
    for op, parallel in operations.items():
        status = "[PARALLEL]" if parallel else "[SEQUENTIAL]"
        print(f"  {op:25s} {status}")
    
    print()
    print("KEY INSIGHT: ALL inner loop operations parallelize across batch items!")
    print("This is why SIMD (vectorization) is so effective here.")
    print()
    print("The only sequential dependency is within each batch item:")
    print("  round N must complete before round N+1 can start.")
    print()
    
    # Verify understanding
    assert all(operations.values()), "Some operations incorrectly marked as sequential"
    print("TEST PASSED: Parallelism opportunities correctly identified.")

test_identify_parallelism()

In [ ]:
def summarize_optimization_path():
    """Summarize all optimizations and their cumulative effect with cycle targets."""
    
    print("THE PATH TO 100X SPEEDUP - CONCRETE MILESTONES")
    print("=" * 60)
    print()
    
    BASELINE = 147734
    
    milestones = [
        ("Scalar baseline", 147734, "1 instruction per cycle"),
        ("Vectorized (8x SIMD)", 7700, "vload/vstore/valu process 8 items"),
        ("+ VLIW packing", 3500, "Multiple ops per instruction"),
        ("+ Software pipelining", 1500, "Overlap loads with hash computation"),
    ]
    
    print("CYCLE MILESTONES (your targets):")
    print("-" * 60)
    
    prev_cycles = None
    for name, cycles, technique in milestones:
        if prev_cycles:
            improvement = prev_cycles / cycles
            print(f"  {name:25s} ~{cycles:,} cycles  ({improvement:.1f}x from previous)")
        else:
            print(f"  {name:25s} {cycles:,} cycles  (starting point)")
        print(f"    Technique: {technique}")
        print()
        prev_cycles = cycles
    
    total_speedup = BASELINE / 1500
    print(f"Total speedup: {total_speedup:.0f}x")
    print()
    
    print("WHICH OPTIMIZATION GIVES WHAT:")
    print("-" * 60)
    print("  Vectorization (8x):     147,734 / 8  = ~18,000 cycles theoretical")
    print("                          Actual ~7,700 (includes overhead)")
    print()
    print("  VLIW packing (~2x):     7,700 / 2   = ~3,500 cycles")
    print("                          Pack loads, ALU ops into fewer instructions")
    print()
    print("  Pipelining (~2.3x):     3,500 / 2.3 = ~1,500 cycles")
    print("                          Hide 12-cycle hash behind 4-cycle loads")
    print()
    
    print("THE MEMORY BOTTLENECK DEFINES THE LIMIT:")
    print("-" * 60)
    print("  Per vector group: 8 scattered tree loads")
    print("  With 2 load slots: 4 cycles minimum for loads")
    print("  512 vector groups total: 512 x 4 = 2,048 cycles minimum")
    print()
    print("  But with perfect pipelining and dual-batch processing:")
    print("  You can approach ~1,500 cycles or even lower!")
    print()
    
    print("Best known results:")
    print("  - Human (2 hrs): 1,790 cycles")
    print("  - Claude Opus 4.5: 1,363 cycles")
    print("  - Target: <1,500 cycles")

summarize_optimization_path()

In [ ]:
def test_memory_layout():
    """Test understanding of memory layout."""
    
    # The memory image has a header, then tree values, then batch data
    # Header format (7 words at addresses 0-6):
    #   [0]: rounds
    #   [1]: n_nodes
    #   [2]: batch_size
    #   [3]: forest_height
    #   [4]: forest_values_p (pointer to tree data)
    #   [5]: inp_indices_p (pointer to batch indices)
    #   [6]: inp_values_p (pointer to batch values)
    
    # For our test case:
    tree_height = 10
    n_nodes = 2**(tree_height + 1) - 1  # 2047
    batch_size = 256
    
    header_size = 7
    forest_values_p = header_size  # = 7
    inp_indices_p = forest_values_p + n_nodes  # = 7 + 2047 = 2054
    inp_values_p = inp_indices_p + batch_size  # = 2054 + 256 = 2310
    
    print("Memory layout for test case:")
    print(f"  Tree height: {tree_height}")
    print(f"  Tree nodes: {n_nodes}")
    print(f"  Batch size: {batch_size}")
    print()
    print("Address ranges:")
    print(f"  Header:        [0, 6]")
    print(f"  Tree values:   [{forest_values_p}, {forest_values_p + n_nodes - 1}]")
    print(f"  Batch indices: [{inp_indices_p}, {inp_indices_p + batch_size - 1}]")
    print(f"  Batch values:  [{inp_values_p}, {inp_values_p + batch_size - 1}]")
    print()
    
    # To load batch items 0-7:
    print("To load batch items 0-7:")
    print(f"  vload indices from address {inp_indices_p}")
    print(f"  vload values from address {inp_values_p}")
    print()
    
    # To load tree value for batch item i:
    print("To load tree value for batch item i:")
    print(f"  address = {forest_values_p} + indices[i]")
    print(f"  (This is why tree loads are scattered!)")
    
    # Verify
    assert n_nodes == 2047
    assert inp_indices_p == 2054
    assert inp_values_p == 2310
    print()
    print("TEST PASSED: Memory layout correctly understood.")

test_memory_layout()

## Summary: The Path to 100x Speedup

Let us recap the optimization strategies and their expected contributions.

In [ ]:
def summarize_optimization_path():
    """Summarize all optimizations and their cumulative effect."""
    
    print("THE PATH TO 100X SPEEDUP")
    print("=" * 50)
    print()
    
    BASELINE = 147734
    current = BASELINE
    
    optimizations = [
        ("Vectorization (SIMD)", 8, 
         "Process 8 batch items per vector operation"),
        ("VLIW packing", 3,
         "Use multiple slots per cycle (12 ALU, 6 vALU, 2 load, 2 store)"),
        ("Loop unrolling", 1.5,
         "Eliminate per-iteration overhead, enable better scheduling"),
        ("Software pipelining", 2,
         "Overlap loads with computation, hide memory latency"),
    ]
    
    print(f"Starting point: {BASELINE:,} cycles")
    print()
    
    for name, speedup, description in optimizations:
        current = current / speedup
        print(f"{name}:")
        print(f"  {description}")
        print(f"  Speedup: {speedup}x -> {int(current):,} cycles")
        print()
    
    total_speedup = BASELINE / current
    print(f"Cumulative speedup: {total_speedup:.0f}x")
    print(f"Estimated final: ~{int(current):,} cycles")
    print()
    print("Note: These multipliers are rough estimates.")
    print("Actual results depend on implementation quality.")
    print()
    print("Best known results:")
    print("  - Human (2 hrs): 1,790 cycles")
    print("  - Claude Opus 4.5: 1,363 cycles")
    print("  - Target: <1,500 cycles")

summarize_optimization_path()

## Next Steps

You now understand:

1. **What the kernel computes**: Tree traversal with hash-based direction selection
2. **Why the baseline is slow**: Uses 1 slot per cycle out of dozens available
3. **How to vectorize**: Process 8 batch items at once with vALU operations
4. **How to pack instructions**: Use VLIW to issue multiple operations per cycle
5. **How to pipeline**: Overlap memory operations with computation

The implementation challenge is translating this understanding into actual code in `perf_takehome.py`. Modify the `KernelBuilder.build_kernel()` method to generate optimized instructions.

Start with vectorization (the biggest win), then add VLIW packing, then work on deeper pipelining.

Good luck!

In [ ]:
# Final summary of key numbers to remember
print("KEY NUMBERS TO REMEMBER:")
print(f"  VLEN = {VLEN}")
print(f"  Batch size = 256 = 32 x {VLEN}")
print(f"  Rounds = 16")
print(f"  Tree nodes = 2047")
print(f"  Scratch = {SCRATCH_SIZE} words")
print()
print("  Baseline: 147,734 cycles")
print("  Target: ~1,500 cycles")
print("  Required speedup: ~100x")